# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

3. Визуализируйте граф вычислений для задачи 2.

## Лабораторная работа 11

In [21]:
import dask.array as da
import h5py
import numpy as np
import pandas as pd

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [82]:
recipes_da = da.from_array(h5py.File('11_dask_array_data/minutes_n_ingredients_full.hdf5')['recipe'], chunks=(100_000, 3))
recipes_da

<ipython-input-82-c7b88e9feed8>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  recipes_da = da.from_array(h5py.File('11_dask_array_data/minutes_n_ingredients_full.hdf5')['recipe'], chunks=(100_000, 3))


dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [13]:
recipes_da_mean = recipes_da[:, 1:].mean(axis=0).compute()
recipes_da_mean

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [14]:
def mean(da_arr):
    return da_arr[:, 1:].mean(axis=0).compute()

In [17]:
%timeit mean(recipes_da)

88.6 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
_recipes_da = recipes_da.rechunk((15_000, 200))
%timeit mean(_recipes_da)

135 ms ± 3.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
_recipes_da = recipes_da.rechunk((100_000, 10_000))
%timeit mean(_recipes_da)

91.5 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


4. Выберите рецепты, время выполнения которых меньше медианного значения

In [20]:
recipes_da[recipes_da[:, 1] < da.median(recipes_da[:, 1], axis=0)].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [81]:
da_unique_with_counts = da.unique(recipes_da[:, 2], return_counts=True)
da.from_array(da_unique_with_counts).T.compute()

array([[     1, 222071],
       [     2, 224158],
       [     3, 229388],
       [     4, 234948],
       [     5, 240720],
       [     6, 244360],
       [     7, 247181],
       [     8, 246747],
       [     9, 246816],
       [    10,  22430],
       [    11,  19094],
       [    12,  15165],
       [    13,  11640],
       [    14,   8284],
       [    15,   6014],
       [    16,   4145],
       [    17,   2793],
       [    18,   1913],
       [    19,   1279],
       [    20,    852],
       [    21,    529],
       [    22,    346],
       [    23,    244],
       [    24,    178],
       [    25,    107],
       [    26,     68],
       [    27,     55],
       [    28,     33],
       [    29,     22],
       [    30,     20],
       [    31,     13],
       [    32,      5],
       [    33,      4],
       [    34,      3],
       [    35,      4],
       [    36,      1],
       [    37,      2],
       [    38,      1],
       [    39,      1],
       [    40,      2],


6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [35]:
q = np.quantile(recipes_da[:, 1].compute(), 0.75)
recipes_da[recipes_da[:, 1] <= q].compute()[:, 1].max()

49

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [42]:
da_arr = da.array([2, 8])
l1 = da.linalg.norm(recipes_da[:, 1:] - da_arr, ord=1, axis=1).argmin()
recipes_da[l1].compute()

array([980542,      2,      8], dtype=int64)

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент